In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import os
if not os.path.exists("./gan_images"):
    os.makedirs("./gan_images")

In [3]:
generator = Sequential()   # 모델 이름을 generator로 정하고 Sequential() 함수를 호출
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2))) 
generator.add(BatchNormalization()) 
generator.add(Reshape((7, 7, 128))) 
generator.add(UpSampling2D()) 
generator.add(Conv2D(64, kernel_size=5, padding='same')) 
generator.add(BatchNormalization()) 
generator.add(Activation(LeakyReLU(0.2))) 
generator.add(UpSampling2D()) 
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh')) 

In [4]:
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6272)              633472    
_________________________________________________________________
batch_normalization (BatchNo (None, 6272)              25088     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 64)        204864    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 14, 14, 64)        0

In [5]:
# 모델 이름을 discriminator로 정하고 Sequential() 함수 호출
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same")) 
discriminator.add(Activation(LeakyReLU(0.2))) 
discriminator.add(Dropout(0.3)) 
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same')) 
discriminator.add(Activation(LeakyReLU(0.2))) 
discriminator.add(Dropout(0.3)) 
discriminator.add(Flatten()) 
discriminator.add(Dense(1, activation='sigmoid')) 
discriminator.compile(loss='binary_crossentropy', optimizer='adam') 
discriminator.trainable = False

In [6]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)             

In [7]:
ginput = Input(shape=(100,)) 
dis_output = discriminator(generator(ginput)) 
gan = Model(ginput, dis_output) 
gan.compile(loss='binary_crossentropy', optimizer='adam') 

In [8]:
gan.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential (Sequential)      (None, 28, 28, 1)         865281    
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 212865    
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [9]:
def gan_train(epoch, batch_size, saving_interval):
    (X_train, _), (_,_) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
    X_train = (X_train - 127.5) / 127.5
    true = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))
    
    for i in range(epoch):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs, true)
        
        noise = np.random.normal(0,1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)
        
        print('epoch:%d\n' % i, ' d-loss:%.4f\n' % d_loss, 'g_loss:%4f\n' % g_loss)
        
        if i % saving_interval == 0:
              #r, c = 5, 5
              noise = np.random.normal(0, 1, (25, 100))
              gen_imgs = generator.predict(noise)

              # Rescale images 0 - 1
              gen_imgs = 0.5 * gen_imgs + 0.5

              fig, axs = plt.subplots(5, 5)
              count = 0
              for j in range(5):
                  for k in range(5):
                      axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                      axs[j, k].axis('off')
                      count += 1
              fig.savefig("gan_images/gan_mnist_%d.png" % i)
        

In [10]:
gan_train(4001,32,200)

epoch:0
  d-loss:0.6797
 g_loss:0.641985

epoch:1
  d-loss:0.4098
 g_loss:0.277274

epoch:2
  d-loss:0.4039
 g_loss:0.043810

epoch:3
  d-loss:0.4312
 g_loss:0.009212

epoch:4
  d-loss:0.4588
 g_loss:0.007423

epoch:5
  d-loss:0.5015
 g_loss:0.023888

epoch:6
  d-loss:0.4906
 g_loss:0.161868

epoch:7
  d-loss:0.4932
 g_loss:0.528622

epoch:8
  d-loss:0.6832
 g_loss:0.433053

epoch:9
  d-loss:0.5508
 g_loss:0.395711

epoch:10
  d-loss:0.4748
 g_loss:0.567014

epoch:11
  d-loss:0.3562
 g_loss:0.934329

epoch:12
  d-loss:0.2680
 g_loss:1.578809

epoch:13
  d-loss:0.1834
 g_loss:2.305599

epoch:14
  d-loss:0.1426
 g_loss:2.864954

epoch:15
  d-loss:0.1449
 g_loss:2.525467

epoch:16
  d-loss:0.1521
 g_loss:2.198279

epoch:17
  d-loss:0.2955
 g_loss:2.431860

epoch:18
  d-loss:0.2837
 g_loss:2.635655

epoch:19
  d-loss:0.6587
 g_loss:1.959051

epoch:20
  d-loss:1.3202
 g_loss:1.393686

epoch:21
  d-loss:0.7863
 g_loss:1.257281

epoch:22
  d-loss:0.6233
 g_loss:1.549463

epoch:23
  d-loss:0.5

epoch:189
  d-loss:0.5969
 g_loss:1.559995

epoch:190
  d-loss:0.4908
 g_loss:1.472535

epoch:191
  d-loss:0.4725
 g_loss:1.334999

epoch:192
  d-loss:0.5461
 g_loss:1.518232

epoch:193
  d-loss:0.5335
 g_loss:1.589404

epoch:194
  d-loss:0.5854
 g_loss:1.685062

epoch:195
  d-loss:0.4509
 g_loss:1.891473

epoch:196
  d-loss:0.3843
 g_loss:1.615516

epoch:197
  d-loss:0.5548
 g_loss:1.402494

epoch:198
  d-loss:0.3507
 g_loss:1.745631

epoch:199
  d-loss:0.4600
 g_loss:1.577330

epoch:200
  d-loss:0.4269
 g_loss:1.577393

epoch:201
  d-loss:0.3568
 g_loss:1.602166

epoch:202
  d-loss:0.4102
 g_loss:1.722096

epoch:203
  d-loss:0.4404
 g_loss:1.447984

epoch:204
  d-loss:0.4978
 g_loss:1.737866

epoch:205
  d-loss:0.3778
 g_loss:1.583636

epoch:206
  d-loss:0.4501
 g_loss:1.767978

epoch:207
  d-loss:0.5681
 g_loss:1.568177

epoch:208
  d-loss:0.3383
 g_loss:1.553287

epoch:209
  d-loss:0.4863
 g_loss:1.481825

epoch:210
  d-loss:0.3630
 g_loss:1.473549

epoch:211
  d-loss:0.3225
 g_los

epoch:376
  d-loss:0.4008
 g_loss:3.322982

epoch:377
  d-loss:0.4686
 g_loss:3.294138

epoch:378
  d-loss:0.6026
 g_loss:2.689880

epoch:379
  d-loss:0.3806
 g_loss:2.605540

epoch:380
  d-loss:0.4336
 g_loss:2.179150

epoch:381
  d-loss:0.3162
 g_loss:2.809786

epoch:382
  d-loss:0.3097
 g_loss:2.888446

epoch:383
  d-loss:0.3452
 g_loss:2.896976

epoch:384
  d-loss:0.3341
 g_loss:2.590159

epoch:385
  d-loss:0.3441
 g_loss:2.776547

epoch:386
  d-loss:0.4211
 g_loss:2.695279

epoch:387
  d-loss:0.2881
 g_loss:3.685185

epoch:388
  d-loss:0.2968
 g_loss:3.813859

epoch:389
  d-loss:0.3836
 g_loss:3.765074

epoch:390
  d-loss:0.2802
 g_loss:2.798418

epoch:391
  d-loss:0.3381
 g_loss:2.740500

epoch:392
  d-loss:0.2873
 g_loss:2.776897

epoch:393
  d-loss:0.4023
 g_loss:2.908131

epoch:394
  d-loss:0.3642
 g_loss:3.487063

epoch:395
  d-loss:0.4764
 g_loss:2.653586

epoch:396
  d-loss:0.4125
 g_loss:1.920632

epoch:397
  d-loss:0.3292
 g_loss:2.431468

epoch:398
  d-loss:0.3884
 g_los

epoch:563
  d-loss:0.1877
 g_loss:3.265754

epoch:564
  d-loss:0.1379
 g_loss:4.258106

epoch:565
  d-loss:0.1940
 g_loss:3.994953

epoch:566
  d-loss:0.2716
 g_loss:3.522971

epoch:567
  d-loss:0.2524
 g_loss:3.447770

epoch:568
  d-loss:0.3065
 g_loss:2.732738

epoch:569
  d-loss:0.1642
 g_loss:3.166558

epoch:570
  d-loss:0.2594
 g_loss:3.453666

epoch:571
  d-loss:0.1377
 g_loss:2.860879

epoch:572
  d-loss:0.3052
 g_loss:3.005589

epoch:573
  d-loss:0.2833
 g_loss:2.646989

epoch:574
  d-loss:0.1869
 g_loss:3.275423

epoch:575
  d-loss:0.3037
 g_loss:3.489523

epoch:576
  d-loss:0.2205
 g_loss:3.290449

epoch:577
  d-loss:0.2848
 g_loss:2.275562

epoch:578
  d-loss:0.1406
 g_loss:2.794996

epoch:579
  d-loss:0.1452
 g_loss:2.827195

epoch:580
  d-loss:0.1621
 g_loss:3.484815

epoch:581
  d-loss:0.2562
 g_loss:3.169567

epoch:582
  d-loss:0.1671
 g_loss:3.271896

epoch:583
  d-loss:0.1349
 g_loss:3.576613

epoch:584
  d-loss:0.1944
 g_loss:3.790803

epoch:585
  d-loss:0.1858
 g_los

epoch:750
  d-loss:0.2862
 g_loss:2.631837

epoch:751
  d-loss:0.2217
 g_loss:2.807347

epoch:752
  d-loss:0.2827
 g_loss:2.357925

epoch:753
  d-loss:0.3063
 g_loss:2.254143

epoch:754
  d-loss:0.3876
 g_loss:2.194739

epoch:755
  d-loss:0.3109
 g_loss:2.048361

epoch:756
  d-loss:0.6000
 g_loss:1.720072

epoch:757
  d-loss:0.5060
 g_loss:2.132560

epoch:758
  d-loss:0.4616
 g_loss:1.898301

epoch:759
  d-loss:0.8042
 g_loss:1.172908

epoch:760
  d-loss:0.5723
 g_loss:1.088999

epoch:761
  d-loss:0.9668
 g_loss:0.886614

epoch:762
  d-loss:0.7605
 g_loss:1.295686

epoch:763
  d-loss:0.7591
 g_loss:1.585418

epoch:764
  d-loss:0.8982
 g_loss:1.283487

epoch:765
  d-loss:0.7136
 g_loss:1.064045

epoch:766
  d-loss:0.8150
 g_loss:0.875235

epoch:767
  d-loss:0.7281
 g_loss:1.019054

epoch:768
  d-loss:0.5722
 g_loss:1.241130

epoch:769
  d-loss:0.6129
 g_loss:1.476766

epoch:770
  d-loss:0.5124
 g_loss:1.611107

epoch:771
  d-loss:0.4473
 g_loss:1.426223

epoch:772
  d-loss:0.5150
 g_los

...

epoch:3317
  d-loss:0.7022
 g_loss:1.157273

epoch:3318
  d-loss:0.5104
 g_loss:1.126503

epoch:3319
  d-loss:0.4293
 g_loss:1.449692

epoch:3320
  d-loss:0.5064
 g_loss:1.667600

epoch:3321
  d-loss:0.5084
 g_loss:1.790226

epoch:3322
  d-loss:0.4742
 g_loss:1.714879

epoch:3323
  d-loss:0.4725
 g_loss:2.043035

epoch:3324
  d-loss:0.4853
 g_loss:1.667102

epoch:3325
  d-loss:0.4421
 g_loss:1.591655

epoch:3326
  d-loss:0.5249
 g_loss:1.384661

epoch:3327
  d-loss:0.5209
 g_loss:1.486776

epoch:3328
  d-loss:0.3532
 g_loss:1.411269

epoch:3329
  d-loss:0.5683
 g_loss:1.718134

epoch:3330
  d-loss:0.3570
 g_loss:2.024497

epoch:3331
  d-loss:0.3715
 g_loss:2.160240

epoch:3332
  d-loss:0.4426
 g_loss:1.944307

epoch:3333
  d-loss:0.4801
 g_loss:1.741345

epoch:3334
  d-loss:0.5074
 g_loss:1.742514

epoch:3335
  d-loss:0.3291
 g_loss:1.925442

epoch:3336
  d-loss:0.3442
 g_loss:2.086520

epoch:3337
  d-loss:0.5363
 g_loss:1.816124

epoch:3338
  d-loss:0.3505
 g_loss:1.834077

epoch:3339

epoch:3500
  d-loss:0.4243
 g_loss:1.363898

epoch:3501
  d-loss:0.4526
 g_loss:1.522994

epoch:3502
  d-loss:0.4877
 g_loss:1.396031

epoch:3503
  d-loss:0.4477
 g_loss:1.609106

epoch:3504
  d-loss:0.5085
 g_loss:1.806349

epoch:3505
  d-loss:0.5283
 g_loss:1.393332

epoch:3506
  d-loss:0.5305
 g_loss:1.699041

epoch:3507
  d-loss:0.5459
 g_loss:1.259493

epoch:3508
  d-loss:0.4660
 g_loss:1.486071

epoch:3509
  d-loss:0.5112
 g_loss:1.208673

epoch:3510
  d-loss:0.4150
 g_loss:1.718347

epoch:3511
  d-loss:0.4443
 g_loss:1.892395

epoch:3512
  d-loss:0.5392
 g_loss:1.641050

epoch:3513
  d-loss:0.5519
 g_loss:1.435693

epoch:3514
  d-loss:0.6079
 g_loss:1.626306

epoch:3515
  d-loss:0.5191
 g_loss:1.644485

epoch:3516
  d-loss:0.4654
 g_loss:1.455957

epoch:3517
  d-loss:0.4139
 g_loss:1.388009

epoch:3518
  d-loss:0.5121
 g_loss:1.591043

epoch:3519
  d-loss:0.4149
 g_loss:1.790147

epoch:3520
  d-loss:0.4165
 g_loss:1.996784

epoch:3521
  d-loss:0.5141
 g_loss:1.692910

epoch:3522

epoch:3683
  d-loss:0.5102
 g_loss:1.641042

epoch:3684
  d-loss:0.3959
 g_loss:1.413389

epoch:3685
  d-loss:0.5856
 g_loss:1.750224

epoch:3686
  d-loss:0.4571
 g_loss:1.810620

epoch:3687
  d-loss:0.5791
 g_loss:1.767226

epoch:3688
  d-loss:0.3996
 g_loss:1.676339

epoch:3689
  d-loss:0.5378
 g_loss:1.401367

epoch:3690
  d-loss:0.4886
 g_loss:1.075751

epoch:3691
  d-loss:0.5738
 g_loss:1.444866

epoch:3692
  d-loss:0.5134
 g_loss:1.186270

epoch:3693
  d-loss:0.7217
 g_loss:1.426356

epoch:3694
  d-loss:0.4983
 g_loss:1.364147

epoch:3695
  d-loss:0.5456
 g_loss:1.681785

epoch:3696
  d-loss:0.6245
 g_loss:1.585192

epoch:3697
  d-loss:0.5294
 g_loss:1.709767

epoch:3698
  d-loss:0.6290
 g_loss:1.324929

epoch:3699
  d-loss:0.5304
 g_loss:1.795013

epoch:3700
  d-loss:0.5153
 g_loss:1.265968

epoch:3701
  d-loss:0.6200
 g_loss:1.281359

epoch:3702
  d-loss:0.7440
 g_loss:1.493994

epoch:3703
  d-loss:0.3935
 g_loss:1.399057

epoch:3704
  d-loss:0.4736
 g_loss:1.350199

epoch:3705

epoch:3866
  d-loss:0.5067
 g_loss:1.986691

epoch:3867
  d-loss:0.5250
 g_loss:1.523530

epoch:3868
  d-loss:0.6705
 g_loss:1.482985

epoch:3869
  d-loss:0.5797
 g_loss:1.297921

epoch:3870
  d-loss:0.5755
 g_loss:1.112185

epoch:3871
  d-loss:0.4307
 g_loss:1.172168

epoch:3872
  d-loss:0.5329
 g_loss:1.423166

epoch:3873
  d-loss:0.4493
 g_loss:1.596811

epoch:3874
  d-loss:0.5846
 g_loss:1.842256

epoch:3875
  d-loss:0.4324
 g_loss:1.900013

epoch:3876
  d-loss:0.4952
 g_loss:1.787025

epoch:3877
  d-loss:0.4732
 g_loss:1.588884

epoch:3878
  d-loss:0.5105
 g_loss:1.542089

epoch:3879
  d-loss:0.5507
 g_loss:1.405807

epoch:3880
  d-loss:0.4056
 g_loss:1.534510

epoch:3881
  d-loss:0.5497
 g_loss:1.411552

epoch:3882
  d-loss:0.5191
 g_loss:1.626395

epoch:3883
  d-loss:0.4209
 g_loss:1.674979

epoch:3884
  d-loss:0.3811
 g_loss:2.134334

epoch:3885
  d-loss:0.4488
 g_loss:2.007707

epoch:3886
  d-loss:0.5675
 g_loss:1.567027

epoch:3887
  d-loss:0.4274
 g_loss:1.811964

epoch:3888

C:\Users\BIT\anaconda3\envs\data-anal-env\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
